In [1]:


import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
    
    


In [2]:
#download all the dependencies that we will need.

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy

!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
#from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
#import folium # map rendering library

#print('Libraries imported.')


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [3]:
#keep this cell, content should be correct.

import pandas as pd 

import requests 

from bs4 import BeautifulSoup 

headers = { 'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36", } 
response = requests.request("GET", "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", headers=headers, verify=False)

#req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 

soup = BeautifulSoup(response.content,'lxml') 

#table = soup.find_all('table')[0]  

#df = pd.read_html(str(table)) 

#neighborhood=pd.DataFrame(df[0]) 


table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)

neighborhoods=pd.DataFrame(table_contents)
neighborhoods['Borough']=neighborhoods['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'en.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [5]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [8]:
!pip install geocoder
import geocoder

latitude=[]
longitude=[]
for code in neighborhoods['PostalCode']:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    #print(code, g.latlng)
    while (g.latlng is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
        #print(code, g.latlng)
    latlng = g.latlng
    latitude.append(latlng[0])
    longitude.append(latlng[1])

In [13]:
# add latitude and longitude to Neighbourhoods Table
neighborhoods['Latitude']=latitude
neighborhoods['Longitude']=longitude
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [16]:
# create map of Toronto using latitude and longitude values
import folium

map_Toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [17]:
NYork_data = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
NYork_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
3,M3B,North York,Don Mills North,43.74923,-79.36186
4,M6B,North York,Glencairn,43.70687,-79.44812


In [21]:
# create map of Downtown Toronto using latitude and longitude values
map_NYork = folium.Map(location=[43.7615, -79.4111], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NYork_data['Latitude'], NYork_data['Longitude'], NYork_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NYork)  
    
map_NYork

In [22]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2' # your Foursquare ID
CLIENT_SECRET = 'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2
CLIENT_SECRET:TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV


In [23]:
#Get the neighborhood's name.
NYork_data.loc[0, 'Neighborhood']

'Parkwoods'

In [25]:
#Get the neighborhood's latitude and longitude values.

neighborhood_latitude = NYork_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = NYork_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = NYork_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


Latitude and longitude values of Parkwoods are 43.75245000000007, -79.32990999999998.


In [26]:
#get the top 100 venues that are in Parkwoods within a radius of 500 meters.


LIMIT = 100 # limit of number of venues returned by Foursquare API


radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    'B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2', 
    'TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV', 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2&client_secret=TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100'

In [36]:
#Send the GET request and examine the resutls

import requests 

url_jason=url+".json()"

headers = { 'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36", } 
results = requests.request("GET", url, headers=headers, verify=False)


#results = requests.get(url).json()
results

C:\Users\wangc8\Anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.foursquare.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [400]>

In [33]:
url_jason

'https://api.foursquare.com/v2/venues/explore?&client_id=B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2&client_secret=TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100.jason()'

In [38]:
requests.request("GET", https://api.foursquare.com/v2/venues/explore?&client_id=B5SV0MV1BJOAXFY0VMICFDUIOS4WWZDUVRDO5UIMSF4TINW2&client_secret=TC0MSC2BI0HF4UEKH5YW01V2HPCX5ALTPZLTHUZHK1VHBTWV&v=20180605&ll=43.75245000000007,-79.32990999999998&radius=500&limit=100.json(), headers=headers, verify=False)

SyntaxError: invalid syntax (<ipython-input-38-31434a2ab5d4>, line 1)